# Computación Científica II
## Laboratorio  #4
    
   Gabriela González Toledo      gabriela.gonzalez@alumnos.usm.cl  201173017-8
   
   Ian Zamorano Escobedo         Ian.zamorano.12@sansano.usm.cl    201273018-k

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

import timeit
from math import ceil, floor
from sympy import *
from sympy.abc import *
from scipy.special import gamma

#%load_ext line_profiler

### Introducción:

En este laboratorio trabajaremos en la resolución del modelo propuesto por FitzHugh-Nagumo, en primera instancia lo resolveremos mediante Diferencias Finitas utilizando dos métodos de discretización(backward/forward).  En la segunda parte resolveremos el modelo mediante el método de las líneas y finalmente probaremos nuestras implementaciones con distintos parámetros para decidir qué metodo  tiene un mejor desempeño.

### Diferencias Finitas en 2D

#### 1. Esquema de discretización forward:

$$
\partial_t u = D_u \Delta u + \lambda u - u^3 - \kappa - \sigma􀀀 v
$$
$$
\tau \partial_t v = D_v \Delta v + u -􀀀 v
$$

$$
u \rightarrow w
$$

$$
v \rightarrow q
$$

$$
w_{i,j,k} = u(x_i,y_j,t_k) = u(i\Delta x, j\Delta y, k \Delta t) \qquad i,j,k \in \{1,2,3, \dots , N\}
$$
$$
q_{i,j,k} = v(x_i,y_j,t_k) = v(i\Delta x, j\Delta y, k \Delta t) \qquad i,j,k \in \{1,2,3, \dots , N\}
$$

$$
w_{xx} = \cfrac{w_{i-1,j,k} - 2 w_{i,j,k} + w_{i+1,j,k}}{\Delta x^{2}}
$$

$$
w_{yy} = \cfrac{w_{i,j-1,k} - 2 w_{i,j,k} + w_{i,j+1,k}}{\Delta y^{2}}
$$

$$
w_{t} = \cfrac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t}
$$



$$
\cfrac{w_{i,j,k+1} - w_{i,j,k}}{\Delta t} = D_w \left( \cfrac{w_{i-1,j,k} - 2 w_{i,j,k} + w_{i+1,j,k}}{\Delta x^{2}} + \cfrac{w_{i,j-1,k} - 2 w_{i,j,k} + w_{i,j+1,k}}{\Delta y^{2}} \right) + \lambda w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma􀀀 q_{i,j,k}
$$

$$
\tau \cfrac{q_{i,j,k+1} - q_{i,j,k}}{\Delta t} = D_q \left( \cfrac{q_{i-1,j,k} - 2 q_{i,j,k} + q_{i+1,j,k}}{\Delta x^{2}} + \cfrac{q_{i,j-1,k} - 2 q_{i,j,k} + q_{i,j+1,k}}{\Delta y^{2}} \right) +  w_{i,j,k} -􀀀  q_{i,j,k}
$$

$$
~
$$

$$
w_{i,j,k+1}  = \Delta t \left[ D_w \left( \cfrac{w_{i-1,j,k} - 2 w_{i,j,k} + w_{i+1,j,k}}{\Delta x^{2}} + \cfrac{w_{i,j-1,k} - 2 w_{i,j,k} + w_{i,j+1,k}}{\Delta y^{2}} \right) + \lambda w_{i,j,k} - w_{i,j,k}^3 - \kappa - \sigma􀀀 q_{i,j,k} \right] + w_{i,j,k}
$$

$$
q_{i,j,k+1}  = \cfrac{ \Delta t \left[ D_q \left( \cfrac{q_{i-1,j,k} - 2 q_{i,j,k} + q_{i+1,j,k}}{\Delta x^{2}} + \cfrac{q_{i,j-1,k} - 2 q_{i,j,k} + q_{i,j+1,k}}{\Delta y^{2}} \right) +  w_{i,j,k} -􀀀  q_{i,j,k} \right]}{\tau } + q_{i,j,k}
$$


#### 2. Desarrollo solve fordward:

In [4]:
from scipy import sparse
def tridiag_forward(dimension, tupla):
    a,b = dimension
    dl = tupla[0] * np.ones(b)
    du = tupla[1] * np.ones(b)
    d0 = tupla[2] * np.ones(b)
    d = np.vstack((dl, du, d0))
    A = sparse.spdiags(d, (-1, 0, 1), a, b)
    A  = A.todense()
    A[:,-1][0,:]=tupla[2]
    A[-1,:][:,0]=tupla[0]
    return A

In [10]:
p = np.array([[1,2,2,1,0], [2,5,4,2,0], [3,4,5,3,0], [2,2,2,2,0]])
T = np.array([[-2,1,0,0,1], [1,-2,1,0,0], [0,1,-2,1,0], [0,0,1,-2,1], [1,0,0,1,-2]])

def solve_forward(u,v, delta_x, delta_y, delta_t, T_f, D_w, D_q, l, tau, sigma, kappa):
    U = np.zeros(((T_f/delta_t + 1),u.shape[0], u.shape[1]), dtype=np.float)
    V = np.zeros(((T_f/delta_t + 1),v.shape[0], v.shape[1]), dtype=np.float)
    T_x = tridiag_forward((u.shape[1],u.shape[1]),(1/delta_x**2,-2/delta_x**2,1/delta_x**2))
    T_y = tridiag_forward((v.shape[1],v.shape[1]),(1/delta_y**2,-2/delta_y**2,1/delta_y**2))
    for i in range(T_f/delta_t ):
        U[i+1] = delta_t*(np.dot(D_w,(np.dot(U[i],T_x) + np.dot(np.transpose(U[i])),T_x)) + l*U[i] - LA.matrix_power(U[i],3) - kappa - np.dot(sigma, V[i]) ) + U[i]
        V[i+1] = (delta_t/tau)*(np.dot(D_q,(np.dot(U[i],T_y) + np.dot(np.transpose(U[i])),T_y))  + U[i] -V[i])
    return U,V

In [12]:
u_0 = np.random.rand(5,5)
v_0 = np.random.rand(5,5)
sol =solve_forward(u_0 , v_0 , 1.0/50 , 1./50, 2, 10 , 0.0003, 0.004, 1.0, 0.2, 1.0, -0.004)
print np.shape(sol)

TypeError: Required argument 'b' (pos 2) not found

### Simulaciones

#### 1. Parámetros

#### 2.Animación

In [38]:
def animate_pattern(sol, time_step=10, frame_step=20):
    #time index
    t_ind = 0
    fig = plt.figure()
    im = plt.imshow(sol[t_ind], cmap=plt.cm.winter)
    plt.xticks([]); plt.yticks([])

    #setting the number of frames
    frames = int(sol.shape[0]/frame_step)
    
    #update function
    def updatefig(t_ind):
        im = plt.imshow(sol[t_ind*frame_step], cmap=plt.cm.winter)
        return im,
    
    #animate it!
    ani = animation.FuncAnimation(fig, updatefig, frames=frames, 
                    interval=time_step, blit=True, repeat=False)
    plt.show()

### Conclusión:

### Referencias: